In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import math
import CRPS.CRPS as pscore
import datetime

pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from normal_evaluation.drbart_evaluation import *

In [2]:
model_name = 'bpic_2017_all_2'
#model_name = 'bpic_2017_all'
log_name = 'train'
with open('../transformed_event_logs/BPIC_2017_all_'+log_name+'.pickle', 'rb') as f:
    test_event_log = pickle.load(f)

test_event_log['case:concept:name'] = test_event_log['case:concept:name'].astype(str)
known_resources = ['User_1','User_10','User_100','User_101','User_102','User_103','User_104','User_105','User_106','User_107','User_108','User_109','User_11','User_110','User_111','User_112','User_113','User_114','User_115','User_116','User_117','User_118','User_119','User_12','User_120','User_121','User_122','User_123','User_124','User_125','User_126','User_127','User_128','User_129','User_13','User_130','User_131','User_132','User_133','User_134','User_135','User_136','User_137','User_138','User_139','User_14','User_140','User_141','User_142','User_143','User_144','User_145','User_146','User_147','User_148','User_149','User_15','User_16','User_17','User_18','User_19','User_2','User_20','User_21','User_22','User_23','User_24','User_25','User_26','User_27','User_28','User_29','User_3','User_30','User_31','User_32','User_33','User_34','User_35','User_36','User_37','User_38','User_39','User_4','User_40','User_41','User_42','User_43','User_44','User_45','User_46','User_47','User_48','User_49','User_5','User_50','User_51','User_52','User_53','User_54','User_55','User_56','User_57','User_58','User_59','User_6','User_60','User_61','User_62','User_63','User_64','User_65','User_66','User_67','User_68','User_69','User_7','User_70','User_71','User_72','User_73','User_74','User_75','User_76','User_77','User_78','User_79','User_8','User_80','User_81','User_82','User_83','User_84','User_85','User_86','User_87','User_88','User_89','User_9','User_90','User_91','User_92','User_93','User_94','User_95','User_96','User_97','User_98','User_99']
known_activities = ['W_Assess potential fraud__ate_abort','W_Assess potential fraud__complete','W_Assess potential fraud__resume','W_Assess potential fraud__schedule','W_Assess potential fraud__start','W_Assess potential fraud__suspend','W_Assess potential fraud__withdraw','W_Call after offers__ate_abort','W_Call after offers__complete','W_Call after offers__resume','W_Call after offers__schedule','W_Call after offers__start','W_Call after offers__suspend','W_Call after offers__withdraw','W_Call incomplete files__ate_abort','W_Call incomplete files__complete','W_Call incomplete files__resume','W_Call incomplete files__schedule','W_Call incomplete files__start','W_Call incomplete files__suspend','W_Complete application__ate_abort','W_Complete application__complete','W_Complete application__resume','W_Complete application__schedule','W_Complete application__start','W_Complete application__suspend','W_Handle leads__complete','W_Handle leads__resume','W_Handle leads__schedule','W_Handle leads__start','W_Handle leads__suspend','W_Handle leads__withdraw','W_Shortened completion __resume','W_Shortened completion __schedule','W_Shortened completion __start','W_Shortened completion __suspend','W_Validate application__ate_abort','W_Validate application__complete','W_Validate application__resume','W_Validate application__schedule','W_Validate application__start','W_Validate application__suspend']

/tmp/ipykernel_2843/1351869102.py:5: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  test_event_log = pickle.load(f)


In [3]:
N = 1000
n_processes = 30
import conduct_evaluation
get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]

likelihoods_A = None
likelihoods_R = None
likelihoods_R_A = None
likelihoods_R_A_S = None
likelihoods_R_A_S_AC = None
likelihoods_R_A_S_RC = None
likelihoods_R_A_S_RC_AC = None
likelihoods_R_A_S_RC_AC_V = None
likelihoods_R_A_S_D = None
likelihoods_R_A_S_D_RC_AC = None

In [4]:
drbart_model_A = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name/',
                     strict_parser=True)
evaluator_A = conduct_evaluation.ConductEvaluation(drbart_model_A, SampleOutcomes_DRBART_Normal_A, {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A = evaluator_A.sample_cases(False, True)

FileNotFoundError: [Errno 2] No such file or directory: '../../../models/bpic_2017_all_2/concept-name/ucuts.json'

In [5]:
np.mean([v.ln() for v in likelihoods_A[0].values()])

TypeError: 'NoneType' object is not subscriptable

In [6]:
np.mean(get_pscores(likelihoods_A))

TypeError: 'NoneType' object is not subscriptable

In [7]:
drbart_model_R_A = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource/',
                     strict_parser=True)
evaluator_R_A = conduct_evaluation.ConductEvaluation(drbart_model_R_A, SampleOutcomes_DRBART_Normal_R_A,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R_A = evaluator_R_A.sample_cases(False, True)

FileNotFoundError: [Errno 2] No such file or directory: '../../../models/bpic_2017_all_2/concept-name_resource/ucuts.json'

In [8]:
np.mean([v.ln() for v in likelihoods_R_A[0].values()])

TypeError: 'NoneType' object is not subscriptable

In [9]:
np.mean(get_pscores(likelihoods_R_A))

TypeError: 'NoneType' object is not subscriptable

In [10]:
drbart_model_R = DRBART(parser_dir = '../../../models/'+model_name+'/resource/',
                     strict_parser=True)
evaluator_R = conduct_evaluation.ConductEvaluation(drbart_model_R, SampleOutcomes_DRBART_Normal_R,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R = evaluator_R.sample_cases(False, True)

ValueError: invalid literal for int() with base 10: ''

In [11]:
np.mean([v.ln() for v in likelihoods_R[0].values()])

TypeError: 'NoneType' object is not subscriptable

In [12]:
np.mean(get_pscores(likelihoods_R))

TypeError: 'NoneType' object is not subscriptable

In [13]:
drbart_model_R_A_S = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day/',
                     strict_parser=True)
evaluator_R_A_S = conduct_evaluation.ConductEvaluation(drbart_model_R_A_S, SampleOutcomes_DRBART_Normal_R_A_S,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R_A_S = evaluator_R_A_S.sample_cases(False, True)

FileNotFoundError: [Errno 2] No such file or directory: '../../../models/bpic_2017_all_2/concept-name_resource_seconds-in-day/ucuts.json'

In [14]:
np.mean([v.ln() for v in likelihoods_R_A_S[0].values()])

TypeError: 'NoneType' object is not subscriptable

In [15]:
np.mean(get_pscores(likelihoods_R_A_S))

TypeError: 'NoneType' object is not subscriptable

In [16]:
drbart_model_R_A_S_AC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_activity-count/',
                     strict_parser=True)
evaluator_R_A_S_AC = conduct_evaluation.ConductEvaluation(drbart_model_R_A_S_AC, SampleOutcomes_DRBART_Normal_R_A_S_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R_A_S_AC = evaluator_R_A_S_AC.sample_cases(False, True)

  0%|                                                                  | 0/24591 [00:00<?, ?it/s]

  0%|                                                      | 1/24591 [00:24<165:38:29, 24.25s/it]

  2%|█▏                                                      | 501/24591 [00:25<14:29, 27.69it/s]

  4%|██▏                                                    | 1001/24591 [00:29<07:42, 51.01it/s]

  6%|███▎                                                   | 1501/24591 [00:34<05:44, 66.95it/s]

...

  4%|█▊                                            | 1001/24591 [01:15<13:52, 28.35it/s]

 61%|██████████████████████████▊                 | 15001/24591 [01:48<00:46, 208.35it/s]

 79%|██████████████████████████████████▉         | 19501/24591 [01:48<00:17, 299.05it/s]

 98%|██████████████████████████████████████████▉ | 24001/24591 [01:50<00:01, 407.77it/s]

100%|████████████████████████████████████████████| 24591/24591 [01:50<00:00, 223.40it/s]

In [17]:
np.mean([v.ln() for v in likelihoods_R_A_S_AC[0].values()])

Decimal('-7.580254308679557752839007259')

In [18]:
np.mean(get_pscores(likelihoods_R_A_S_AC))

np.float64(12898749074535.953)

In [19]:
drbart_model_R_A_S_RC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_resource-count/',
                     strict_parser=True)
evaluator_R_A_S_RC = conduct_evaluation.ConductEvaluation(drbart_model_R_A_S_RC, SampleOutcomes_DRBART_Normal_R_A_S_RC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                        'known_resources' : known_resources
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R_A_S_RC = evaluator_R_A_S_RC.sample_cases(False, True)

  0%|                                                         | 0/24591 [00:00<?, ?it/s]

  0%|                                             | 1/24591 [00:24<164:07:27, 24.03s/it]

  2%|▉                                              | 501/24591 [00:34<21:47, 18.43it/s]

  4%|█▊                                            | 1001/24591 [00:34<08:51, 44.36it/s]

  6%|██▊                                           | 1501/24591 [00:35<05:05, 75.68it/s]

...

 61%|██████████████████████████▊                 | 15001/24591 [01:53<01:15, 126.56it/s]

 65%|████████████████████████████▋               | 16001/24591 [01:53<00:56, 150.85it/s]

 79%|██████████████████████████████████▉         | 19501/24591 [01:55<00:19, 260.59it/s]

 89%|███████████████████████████████████████▎    | 22001/24591 [01:55<00:06, 376.46it/s]

100%|████████████████████████████████████████████| 24591/24591 [01:55<00:00, 212.32it/s]

In [20]:
np.mean([v.ln() for v in likelihoods_R_A_S_RC[0].values()])

Decimal('-7.067323249060054494613653804')

In [21]:
np.mean(get_pscores(likelihoods_R_A_S_RC))

np.float64(2.544541794803543e+20)

In [22]:
drbart_model_R_A_S_RC_AC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_activity-count_resource_count/',
                     strict_parser=True)
evaluator_R_A_S_RC_AC = conduct_evaluation.ConductEvaluation(drbart_model_R_A_S_RC_AC, SampleOutcomes_DRBART_Normal_R_A_S_RC_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                        'known_resources' : known_resources,
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R_A_S_RC_AC = evaluator_R_A_S_RC_AC.sample_cases(False, True)

  0%|                                                         | 0/24591 [00:00<?, ?it/s]

  0%|                                             | 1/24591 [00:22<150:26:27, 22.02s/it]

  2%|▉                                              | 501/24591 [00:27<16:45, 23.95it/s]

  4%|█▊                                            | 1001/24591 [00:32<09:07, 43.09it/s]

  6%|██▊                                           | 1501/24591 [00:37<06:37, 58.12it/s]

...

 45%|███████████████████▋                        | 11001/24591 [01:16<00:18, 748.77it/s]

 61%|██████████████████████████▊                 | 15001/24591 [01:47<00:53, 180.26it/s]

 65%|████████████████████████████▋               | 16001/24591 [01:48<00:42, 202.58it/s]

 73%|████████████████████████████████▏           | 18001/24591 [01:48<00:24, 266.04it/s]

100%|████████████████████████████████████████████| 24591/24591 [01:48<00:00, 225.69it/s]

In [23]:
np.mean([v.ln() for v in likelihoods_R_A_S_RC_AC[0].values()])

Decimal('-11.17476549296487704979289434')

In [24]:
np.mean(get_pscores(likelihoods_R_A_S_RC_AC))

np.float64(3493670151538.497)

In [25]:
drbart_model_R_A_S_RC_AC_V = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_activity-count_resource_count_amount/',
                     strict_parser=True)
evaluator_R_A_S_RC_AC_V = conduct_evaluation.ConductEvaluation(drbart_model_R_A_S_RC_AC_V, SampleOutcomes_DRBART_Normal_R_A_S_RC_AC_V,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                        'amount' : 'case.RequestedAmount_start',
                                                        'known_resources' : known_resources,
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R_A_S_RC_AC_V = evaluator_R_A_S_RC_AC_V.sample_cases(False, True)

  0%|                                                         | 0/24591 [00:00<?, ?it/s]

  0%|                                             | 1/24591 [00:20<138:01:54, 20.21s/it]

  2%|▉                                              | 501/24591 [00:25<15:20, 26.16it/s]

  4%|█▊                                            | 1001/24591 [00:29<08:00, 49.11it/s]

  6%|██▊                                           | 1501/24591 [00:34<06:14, 61.58it/s]

...

 98%|███████████████████████████████████████████▉ | 24001/24591 [04:27<00:07, 73.83it/s]

100%|█████████████████████████████████████████████| 24591/24591 [04:27<00:00, 92.09it/s]

  0%|                                                         | 0/24591 [00:00<?, ?it/s]

  0%|                                                         | 0/24591 [00:05<?, ?it/s]

TypeError: '<' not supported between instances of 'NoneType' and 'float'

In [26]:
np.mean([v.ln() for v in likelihoods_R_A_S_RC_AC_V[0].values()])

TypeError: 'NoneType' object is not subscriptable

In [27]:
np.mean(get_pscores(likelihoods_R_A_S_RC_AC_V))

TypeError: 'NoneType' object is not subscriptable

In [28]:
drbart_model_R_A_S_D = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_day-of-week/',
                     strict_parser=True)
evaluator_R_A_S_D = conduct_evaluation.ConductEvaluation(drbart_model_R_A_S_D, SampleOutcomes_DRBART_Normal_R_A_S_D,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R_A_S_D = evaluator_R_A_S_D.sample_cases(False, True)

  0%|                                                         | 0/24591 [00:00<?, ?it/s]

  0%|                                             | 1/24591 [00:24<165:06:35, 24.17s/it]

  2%|▉                                              | 501/24591 [00:24<13:49, 29.05it/s]

  4%|█▊                                            | 1001/24591 [00:29<07:44, 50.78it/s]

  6%|██▊                                           | 1501/24591 [00:33<05:41, 67.53it/s]

...

 43%|███████████████████████████████▋                                          | 10525/24591 [01:19<00:34, 406.48it/s]

 61%|█████████████████████████████████████████████▏                            | 15001/24591 [01:48<01:08, 140.33it/s]

 63%|██████████████████████████████████████████████▋                           | 15501/24591 [01:48<01:00, 150.11it/s]

 75%|███████████████████████████████████████████████████████▋                  | 18501/24591 [01:50<00:24, 247.35it/s]

100%|██████████████████████████████████████████████████████████████████████████| 24591/24591 [01:50<00:00, 223.46it/s]

In [29]:
np.mean([v.ln() for v in likelihoods_R_A_S_D[0].values()])

Decimal('-4.290403024569822289820013456')

In [30]:
np.mean(get_pscores(likelihoods_R_A_S_D))

np.float64(855621.0025743234)

In [31]:
drbart_model_R_A_S_D_RC_AC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_day-of-week_activity-count_resoure-count/',
                     strict_parser=True)
evaluator_R_A_S_D_RC_AC = conduct_evaluation.ConductEvaluation(drbart_model_R_A_S_D_RC_AC, SampleOutcomes_DRBART_Normal_R_A_S_D_RC_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                        'known_resources' : known_resources,
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R_A_S_D_RC_AC = evaluator_R_A_S_D_RC_AC.sample_cases(False, True)

  0%|                                                                                       | 0/24591 [00:00<?, ?it/s]

  0%|                                                                           | 1/24591 [00:22<152:21:40, 22.31s/it]

  2%|█▌                                                                           | 501/24591 [00:26<16:09, 24.85it/s]

  4%|███                                                                         | 1001/24591 [00:35<10:27, 37.57it/s]

  6%|████▋                                                                       | 1501/24591 [00:36<06:02, 63.62it/s]

...

 57%|██████████████████████████████████████████▏                               | 14001/24591 [01:16<00:19, 535.23it/s]

 61%|█████████████████████████████████████████████▏                            | 15001/24591 [01:39<01:01, 156.27it/s]

 63%|██████████████████████████████████████████████▋                           | 15501/24591 [01:42<00:58, 156.44it/s]

 65%|████████████████████████████████████████████████▏                         | 16001/24591 [01:43<00:51, 167.35it/s]

100%|██████████████████████████████████████████████████████████████████████████| 24591/24591 [01:43<00:00, 237.22it/s]

In [32]:
np.mean([v.ln() for v in likelihoods_R_A_S_D_RC_AC[0].values()])

Decimal('-10.93853373343194776227653078')

In [33]:
np.mean(get_pscores(likelihoods_R_A_S_D_RC_AC))

np.float64(2180834277123.2483)

In [34]:
results = {
    'drbart_model_A' : likelihoods_A,
    'drbart_model_R' : likelihoods_R,
    'drbart_model_R_A' : likelihoods_R_A,
    'drbart_model_R_A_S' : likelihoods_R_A_S,
    'drbart_model_R_A_S_AC' : likelihoods_R_A_S_AC,
    'drbart_model_R_A_S_RC' : likelihoods_R_A_S_RC,
    'drbart_model_R_A_S_RC_AC' : likelihoods_R_A_S_RC_AC,
    'drbart_model_R_A_S_RC_AC_V' : likelihoods_R_A_S_RC_AC_V,
    'drbart_model_R_A_S_D' : likelihoods_R_A_S_D,
    'drbart_model_R_A_S_D_RC_AC' : likelihoods_R_A_S_D_RC_AC
}
with open('./'+model_name+'_dr_bart_evaluation_'+log_name+'.pickle', 'wb') as handle:
    pickle.dump(results, handle)